In [1]:
import pykat
import pylab as pl
import pandas
pandas.set_eng_float_format(accuracy=3, use_eng_prefix=True)

C:\Users\tvo88\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


                                              ..-
    PyKat 1.1.277         _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,
   `^-....____:   +.      www.gwoptics.org/pykat



In [2]:
katfile = 'aligo_IFO_maxtem2.kat'
basekat = pykat.finesse.kat()
basekat.loadKatFile(katfile)

# Add beam parameter detectors
basekat.parseCommands("""
bp wx x w nYBS
bp wy y w nYBS
bp rcx x r nYBS
bp rcy y r nYBS
""")

# Don't need these things so removing them
basekat.removeLine("startnode")
basekat.removeBlock('locks',True)
basekat.removeBlock('ASWFS',True)
basekat.removeBlock('REFLWFS',True)
basekat.verbose=False
basekat.parseCommands('attr OM3 Rc 1e12')

# Get initial beam parameters at BS using YARM cav as reference
kat=basekat.deepcopy()
kat.noxaxis = True
for cav in kat.getAll(pykat.commands.cavity):
    cav.enabled=False

kat.cavYARM.enabled=True
out = kat.run()

wx_init=out['wx']
wy_init=out['wy']
sx_init=1/out['rcx']
sy_init=1/out['rcy']

# Set amount by which optic power should be changed to compute beam parameter slopes (in Diopters)
dP = 1e-9

# List of potential actuator optics, short names for plotting etc.
# Please add to this list Antonio...
acts=['PRM','PR2','PR3','SRM','SR2','SR3','ITMTL','ITM','ETM','OM1','OM2','OM3']

# Actual names of components in Finesse model that are adjusted for each actuator
# Please add to this list Antonio...
act_names = {}
act_names['PRM'] = 'PRMHR'
act_names['PR2'] = 'PR2'
act_names['PR3'] = 'PR3'
act_names['SRM'] = 'SRMHR'
act_names['SR2'] = 'SR2'
act_names['SR3'] = 'SR3'
act_names['ITMTL'] = 'ITMXTL'
#act_names['ITMYTL'] = 'ITMYTL'
#act_names['ITMX'] = 'ITMXHR'
act_names['ITM'] = 'ITMXHR'
#act_names['ITMY'] = 'ITMYHR'
#act_names['ETMX'] = 'ETMXHR'
act_names['ETM'] = 'ETMXHR'
#act_names['ETMY'] = 'ETMYHR'
act_names['OM1'] = 'OM1'
act_names['OM2'] = 'OM2'
act_names['OM3'] = 'OM3'


# Curved mirror or lens
# Please add to this list Antonio...

act_types = {}

for act in acts:
    act_types[act] = 'mirror'
    
# act_types['ITMXTL'] = 'lens'
# act_types['ITMYTL'] = 'lens'
act_types['ITMTL'] = 'lens'

# Diameters of optics, in case we want to use these for normalizing by optic stress, dissipated power or something.
# Please add to this list Antonio...
act_diams = {}
act_diams['PRM'] = 0.15
act_diams['ITMTL'] = 0.34
act_diams['ITM'] = 0.34

# Find the initial values of actuators, in Diopters (yes even curved mirrors)
act_initvals = {}
for act in acts:
    
    # Get the relevant component object from the kat object
    fin_act_str='fin_act = basekat.'+act_names[act]
    exec(fin_act_str)
    
    # For mirrors, set the initial value in terms of optical power (2/Rc)
    if act_types[act]=='mirror':
        act_initvals[act]=2/fin_act.Rc.value
    
    # For lenses, set the initial value in terms of lens power (1/f)
    elif act_types[act]=='lens':
        act_initvals[act]=1/fin_act.f.value

# Generate xaxis commands for each actuator
act_commands = {}

for act in acts:
    if act_types[act]=='mirror':
        act_commands[act]= """
        xaxis {mname} Rcx lin {start} {stop} 1
        put {mname} Rcy $x1
        """.format(mname=act_names[act], start=(2/act_initvals[act]), stop=(2/(act_initvals[act]+dP)))
        
    elif act_types[act]=='lens':
        act_commands[act]= """
        xaxis {lname} p lin {start} {stop} 1
        """.format(lname=act_names[act], start=(act_initvals[act]), stop=(act_initvals[act]+dP))


C:\Users\tvo88\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: loadKatFile() depreciated, use load(...).
  This is separate from the ipykernel package so we can avoid doing imports until


Parsing `set PRCL_err POP_f1_I re` into pykat object not implemented yet, added as extra line.
Parsing `set MICH_err POP_f2_Q re` into pykat object not implemented yet, added as extra line.
Parsing `set CARM_err REFL_f1_I re` into pykat object not implemented yet, added as extra line.
Parsing `set SRCL_err REFL_f2_I re` into pykat object not implemented yet, added as extra line.
Parsing `set OMC_DC P_DC_OMC re` into pykat object not implemented yet, added as extra line.


C:\Users\tvo88\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: parseCommands depreciated, use parse.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\tvo88\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: parseCommands depreciated, use parse.


In [3]:
# Set up a list of cavity DOF names (for plots and matrices), defocus and size for each cavity

#cavnames = ['cavPRX','cavPRY','cavSRX','cavSRY','cavXARM','cavYARM','cavIMC','cavOMC']
cavnames = ['cavPRX','cavSRX','cavXARM','cavOMC']

cavDOFs = []

for cavname in cavnames:
    print(cavname)
    if (cavname == 'cavPRX') or (cavname == 'cavSRX'):
        cavDOFs.append(cavname[3:len(cavname)-1]+'C w')
        cavDOFs.append(cavname[3:len(cavname)-1]+'C S')
    elif cavname == 'cavXARM':
        cavDOFs.append('ARMs w')
        cavDOFs.append('ARMs S')
    else:
        cavDOFs.append(cavname[3:]+' w')
        cavDOFs.append(cavname[3:]+' S')

# intialize dataframes for storing the actuation matrix values, one for x-axis, one for y-axis

act_matrix_x = pandas.DataFrame(columns=cavDOFs, index=acts)
act_matrix_y = pandas.DataFrame(columns=cavDOFs, index=acts)

cavPRX
cavSRX
cavXARM
cavOMC


In [14]:
# specify a subset of the full list of actuators here if needed
#acts=['PRM','PR2','PR3','SRM','SR2','SR3','ITMXTL','ITMYTL','ITMX','ITMY','ETMX','ETMY','OM1','OM2','OM3']

k=2*pl.pi/1064e-9

# loop over actuators
for act in acts:
    kat=basekat.deepcopy()
    # apply actuator specific xaxis commands
    kat.parse(act_commands[act])
    
    # loop through all cavities
    for cav in cavnames:
        
        # First turn off all cavity commands
        for allcavs in kat.getAll(pykat.commands.cavity):
            allcavs.enabled = False
        
        # Now turn on just the one we're looking at in this loop
        katcavstr='katcav=kat.'+cav
        exec(katcavstr)
        katcav.enabled = True
    
        # Run simulation
        out=kat.run()
        
        # Calculate slopes of cavity parameters w.r.t. optic defocus change
        wx_slope = (out['wx'][1]-out['wx'][0])/dP
        sx_slope = (1/out['rcx'][1]-1/out['rcx'][0])/dP
        wy_slope = (out['wy'][1]-out['wy'][0])/dP
        sy_slope = (1/out['rcy'][1]-1/out['rcy'][0])/dP

        # Place the calculated slopes in the actuation matrix, normalizing by the HG20 coupling factor
        
        if (katcav.name == 'cavPRX') or (katcav.name == 'cavSRX'):
            cavname = katcav.name[3:len(katcav.name)-1]+'C'
        elif katcav.name == 'cavXARM':
            cavname='ARMs'
        else:
            cavname=katcav.name[3:]

        act_matrix_x[cavname+' w'][act] = wx_slope/wx_init
        act_matrix_x[cavname+' S'][act] = sx_slope*wx_init**2*k/4
        act_matrix_y[cavname+' w'][act] = wy_slope/wy_init
        act_matrix_y[cavname+' S'][act] = sy_slope*wy_init**2*k/4


In [5]:
# Print actuation matrix. 

# There are fancier ways to do this, with nice formatting and making a LaTeX table etc., 
# but for now I just leave it like this. Units are currently all 1/D, or more specifically:
# For beam size columns, units are [change in beam size] / [intial beam size] per Diopter on actuator
# For beam defocus columns, units are [change in beam defocus] / [intial beam defocus] per Diopter on actuator
pandas.set_eng_float_format(accuracy=2, use_eng_prefix=True)

In [6]:
wx_slope/wx_init

-0.22238236978941445

In [15]:
act_matrix_x

,PRC w,PRC S,SRC w,SRC S,ARMs w,ARMs S,OMC w,OMC S
PRM,4.54,5.90m,0.00,0.00,0.00,0.00,0.00,0.00
PR2,-45.00,-58.46m,0.00,0.00,0.00,0.00,0.00,0.00
PR3,-3.32k,-4.32,0.00,0.00,0.00,0.00,0.00,0.00
SRM,0.00,0.00,-5.10,-6.29m,0.00,0.00,875.20m,1.11
SR2,0.00,0.00,-128.89,-159.87m,0.00,0.00,-2.79,-99.51
SR3,0.00,0.00,-5.46k,-6.77,0.00,0.00,-19.56,-4.29k
ITMTL,-3.19k,4.14k,-5.25k,4.14k,-5.01,4.15k,0.00,0.00
ITM,-2.31k,3.00k,-3.81k,3.00k,-1.83k,3.00k,0.00,0.00
ETM,0.00,0.00,0.00,0.00,3.77k,4.68,0.00,0.00
OM1,0.00,0.00,0.00,0.00,0.00,0.00,163.75m,551.87m


In [16]:
act_matrix_y

,PRC w,PRC S,SRC w,SRC S,ARMs w,ARMs S,OMC w,OMC S
PRM,5.09,6.03m,0.00,0.00,0.00,0.00,0.00,0.00
PR2,-52.02,-61.71m,0.00,0.00,0.00,0.00,0.00,0.00
PR3,-3.85k,-4.57,0.00,0.00,0.00,0.00,0.00,0.00
SRM,0.00,0.00,-6.87,-6.52m,0.00,0.00,869.93m,1.11
SR2,0.00,0.00,-178.70,-169.26m,0.00,0.00,-2.78,-99.47
SR3,0.00,0.00,-7.59k,-7.19,0.00,0.00,-19.51,-4.29k
ITMTL,-3.70k,4.14k,-7.31k,4.14k,-5.02,4.15k,0.00,0.00
ITM,-2.68k,3.00k,-5.30k,3.00k,-1.83k,3.00k,0.00,0.00
ETM,0.00,0.00,0.00,0.00,3.77k,4.70,0.00,0.00
OM1,0.00,0.00,0.00,0.00,0.00,0.00,171.73m,557.07m


In [17]:
OM3cpx=act_matrix_x['OMC w']['OM3']-1j*act_matrix_x['OMC S']['OM3']
OM2cpx=act_matrix_x['OMC w']['OM2']-1j*act_matrix_x['OMC S']['OM2']
OM1cpx=act_matrix_x['OMC w']['OM1']-1j*act_matrix_x['OMC S']['OM1']
SRMcpx=-act_matrix_x['OMC w']['SRM']+1j*act_matrix_x['OMC S']['SRM'] #have to switch signs because curvature sign is different
SR2cpx=act_matrix_x['OMC w']['SR2']-1j*act_matrix_x['OMC S']['SR2']
SR3cpx=act_matrix_x['OMC w']['SR3']-1j*act_matrix_x['OMC S']['SR3']

OM3Gouy=0.5*(pl.angle(OM3cpx*pl.exp(-1j*pl.pi*1/2)))
OM2Gouy=0.5*(pl.angle(OM2cpx*pl.exp(-1j*pl.pi*1/2)))
OM1Gouy=0.5*(pl.angle(OM1cpx*pl.exp(-1j*pl.pi*1/2)))
SRMGouy=0.5*(pl.angle(SRMcpx*pl.exp(-1j*pl.pi*1/2)))
SR2Gouy=0.5*(pl.angle(SR2cpx*pl.exp(-1j*pl.pi*1/2)))
SR3Gouy=0.5*(pl.angle(SR3cpx*pl.exp(-1j*pl.pi*1/2)))

print('OM3 Gouy phase calculated from actuation matrix = '+str(OM3Gouy*180/pl.pi+180)+' deg')
print('OM2 Gouy phase calculated from actuation matrix = '+str(OM2Gouy*180/pl.pi+180)+' deg')
print('OM1 Gouy phase calculated from actuation matrix = '+str(OM1Gouy*180/pl.pi+180)+' deg')
print('SRM Gouy phase calculated from actuation matrix = '+str(SRMGouy*180/pl.pi)+' deg')
print('SR2 Gouy phase calculated from actuation matrix = '+str(SR2Gouy*180/pl.pi)+' deg')
print('SR3 Gouy phase calculated from actuation matrix = '+str(SR3Gouy*180/pl.pi)+' deg\n')


kat=basekat.deepcopy()
kat.noxaxis=True
for cav in kat.getAll(pykat.commands.cavity):
    cav.enabled=False

kat.cavOMC.enabled=True

kat.parseCommands("""
gouy OM3gouyx x BSsub2 ls3 ls2 ls1 SRMsub sSRM_OM1 sOM1_OM2 sOM2_OM3
gouy OM2gouyx x BSsub2 ls3 ls2 ls1 SRMsub sSRM_OM1 sOM1_OM2
gouy OM1gouyx x BSsub2 ls3 ls2 ls1 SRMsub sSRM_OM1
gouy SRMgouyx x BSsub2 ls3 ls2 ls1
gouy SR2gouyx x BSsub2 ls3 ls2
gouy SR3gouyx x BSsub2 ls3""")
out=kat.run()

print('OM3 Gouy phase direct from Finesse file = '+str(out['OM3gouyx'])+' deg')
print('OM2 Gouy phase direct from Finesse file = '+str(out['OM2gouyx'])+' deg')
print('OM1 Gouy phase direct from Finesse file = '+str(out['OM1gouyx'])+' deg')
print('SRM Gouy phase direct from Finesse file = '+str(out['SRMgouyx'])+' deg')
print('SR2 Gouy phase direct from Finesse file = '+str(out['SR2gouyx'])+' deg')
print('SR3 Gouy phase direct from Finesse file = '+str(out['SR3gouyx'])+' deg')


OM3 Gouy phase calculated from actuation matrix = 193.06318535941534 deg
OM2 Gouy phase calculated from actuation matrix = 165.06881308224646 deg
OM1 Gouy phase calculated from actuation matrix = 98.26347426624574 deg
SRM Gouy phase calculated from actuation matrix = 19.072221085080947 deg
SR2 Gouy phase calculated from actuation matrix = 0.803606203328307 deg
SR3 Gouy phase calculated from actuation matrix = 0.13050652462790163 deg

OM3 Gouy phase direct from Finesse file = 193.246192083736 deg
OM2 Gouy phase direct from Finesse file = 164.868739843951 deg
OM1 Gouy phase direct from Finesse file = 98.3905309070352 deg
SRM Gouy phase direct from Finesse file = 19.2971440623399 deg
SR2 Gouy phase direct from Finesse file = 0.816667105736422 deg
SR3 Gouy phase direct from Finesse file = 0.132628777933525 deg


C:\Users\tvo88\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: UserWarning: parseCommands depreciated, use parse.


In [18]:
f = act_matrix_x.to_latex()

In [19]:
print(act_matrix_x.to_latex(float_format=lambda x: '%.3f' % x))

\begin{tabular}{lllllllll}
\toprule
{} &     PRC w &    PRC S &     SRC w &    SRC S &    ARMs w &   ARMs S &   OMC w &     OMC S \\
\midrule
PRM   &     4.537 &    0.006 &     0.000 &    0.000 &     0.000 &    0.000 &   0.000 &     0.000 \\
PR2   &   -45.004 &   -0.058 &     0.000 &    0.000 &     0.000 &    0.000 &   0.000 &     0.000 \\
PR3   & -3321.891 &   -4.315 &     0.000 &    0.000 &     0.000 &    0.000 &   0.000 &     0.000 \\
SRM   &     0.000 &    0.000 &    -5.097 &   -0.006 &     0.000 &    0.000 &   0.875 &     1.114 \\
SR2   &     0.000 &    0.000 &  -128.889 &   -0.160 &     0.000 &    0.000 &  -2.792 &   -99.513 \\
SR3   &     0.000 &    0.000 & -5460.964 &   -6.774 &     0.000 &    0.000 & -19.564 & -4294.447 \\
ITMTL & -3188.263 & 4142.666 & -5254.308 & 4140.289 &    -5.006 & 4146.801 &   0.000 &     0.000 \\
ITM   & -2310.433 & 3002.047 & -3807.633 & 3000.325 & -1827.435 & 3002.782 &   0.000 &     0.000 \\
ETM   &     0.000 &    0.000 &     0.000 &    0.000 &  377

In [20]:
print(act_matrix_y.to_latex(float_format=lambda x: '%.3f' % x))

\begin{tabular}{lllllllll}
\toprule
{} &     PRC w &    PRC S &     SRC w &    SRC S &    ARMs w &   ARMs S &   OMC w &     OMC S \\
\midrule
PRM   &     5.092 &    0.006 &     0.000 &    0.000 &     0.000 &    0.000 &   0.000 &     0.000 \\
PR2   &   -52.019 &   -0.062 &     0.000 &    0.000 &     0.000 &    0.000 &   0.000 &     0.000 \\
PR3   & -3852.041 &   -4.569 &     0.000 &    0.000 &     0.000 &    0.000 &   0.000 &     0.000 \\
SRM   &     0.000 &    0.000 &    -6.870 &   -0.007 &     0.000 &    0.000 &   0.870 &     1.105 \\
SR2   &     0.000 &    0.000 &  -178.704 &   -0.169 &     0.000 &    0.000 &  -2.783 &   -99.469 \\
SR3   &     0.000 &    0.000 & -7589.021 &   -7.189 &     0.000 &    0.000 & -19.509 & -4292.095 \\
ITMTL & -3700.089 & 4142.316 & -7309.182 & 4139.780 &    -5.022 & 4146.698 &   0.000 &     0.000 \\
ITM   & -2681.338 & 3001.794 & -5296.743 & 2999.956 & -1827.470 & 3002.700 &   0.000 &     0.000 \\
ETM   &     0.000 &    0.000 &     0.000 &    0.000 &  377